In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')

In [3]:
df_train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398648 entries, 0 to 398647
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   waktu_setempat    398648 non-null  object 
 1   id_jalan          398648 non-null  int64  
 2   id_titik_mulai    398648 non-null  int64  
 3   id_titik_akhir    398648 non-null  int64  
 4   rerata_kecepatan  398648 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 15.2+ MB


## Timestamp formating

In [5]:
# Slice and format datatime 

df_train['waktu_setempat'] = df_train['waktu_setempat'].str.slice(0, 19)
df_train['waktu_setempat'] = pd.to_datetime(df_train['waktu_setempat'])

df_test['waktu_setempat'] = df_test['waktu_setempat'].str.slice(0, 19)
df_test['waktu_setempat'] = pd.to_datetime(df_test['waktu_setempat'])

In [6]:
df_train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan
0,2020-02-01 01:00:00,691007296,21390008,1425033102,29.126
1,2020-02-01 01:00:00,47010584,1677092762,579493410,46.576
2,2020-02-01 01:00:00,22932408,26486694,1930267566,36.587
3,2020-02-01 01:00:00,142479648,1111592522,3775231113,34.063
4,2020-02-01 01:00:00,8504977,5940503398,5940503394,38.336


## External Data Merger

In [8]:
static_features = pd.read_csv('dataset/external_data.csv')
static_features.head()

,id_jalan,lanes,maxspeed,highway
0,691007296,3.0,30,trunk
1,47010584,NaN,30,primary
2,22932408,NaN,20,secondary
3,142479648,3.0,20,primary
4,8504977,2.0,30,primary


In [9]:
df_train = df_train.merge(static_features, on='id_jalan', how='left')
df_test = df_test.merge(static_features, on='id_jalan', how='left')

## Ide: Membuat id baru untuk tiap id_jalan - id_titik_mulai - id_titik_akhir yang unique

In [10]:
list_id = ['id_jalan', 'id_titik_mulai', 'id_titik_akhir']

for i in list_id:
    df_train[i] = df_train[i].astype('str')
    df_test[i] = df_test[i].astype('str')

In [11]:
df_train['new_id'] = df_train['id_jalan'] + '_' + df_train['id_titik_mulai'] + '_' + df_train['id_titik_akhir']
df_train = df_train.drop(list_id, axis = 1)

df_test['new_id'] = df_test['id_jalan'] + '_' + df_test['id_titik_mulai'] + '_' + df_test['id_titik_akhir']
df_test = df_test.drop(list_id, axis = 1)

In [14]:
df_train.head()

,waktu_setempat,rerata_kecepatan,lanes,maxspeed,highway,new_id
0,2020-02-01 01:00:00,29.126,3.0,30,trunk,691007296_21390008_1425033102
1,2020-02-01 01:00:00,46.576,NaN,30,primary,47010584_1677092762_579493410
2,2020-02-01 01:00:00,36.587,NaN,20,secondary,22932408_26486694_1930267566
3,2020-02-01 01:00:00,34.063,3.0,20,primary,142479648_1111592522_3775231113
4,2020-02-01 01:00:00,38.336,2.0,30,primary,8504977_5940503398_5940503394


In [15]:
len(df_train.new_id.unique())

934

Saat ini, terdapat 934 time series yang berbeda yang siap untuk diproses

### Save into csv

In [16]:
df_train.to_csv(r'dataset/train_prepared.csv', index=False)
df_test.to_csv(r'dataset/test_prepared.csv', index=False)